In [ ]:
### user input (and autofill) for limited number of fruits/veggies/grains/nuts

### (A) user input for "where are you" (state) - future development, for now just go to centroid of the U.S.

### (B) user input for "where is your food from" (input country of origin, if U.S., then State, then city)

### (C) calculate distance between centroids (A and B)

### output: C

### (D) multiply distance by the amount of CO2 is generated by air, land, and 
### water travel (produce a range of possible emissions)

### output: D

### provide water use assuming that the net weight of given produce is 1 pound. Divide blue and total water use by 2000 
### because it's in metric tons and then multiply to generate how many gallons & liters are used